# Inverted Tilesheet for Sheep Planet

This creates latex figures that act as tilesheets for each of the planets.

First we load the planet textures from the source code.

In [10]:
charset_lines = """
upperPlanetSurfaceCharset                                                             
        .BYTE $00,$00,$20,$00,$8A,$AA,$00,$AA   ;.BYTE $00,$00,$20,$00,$8A,$AA,$00,$AA
                                                ; CHARACTER $40                       
                                                ; 00000000                            
                                                ; 00000000                            
                                                ; 00100000     *                      
                                                ; 00000000                            
                                                ; 10001010   *   * *                  
                                                ; 10101010   * * * *                  
                                                ; 00000000                            
                                                ; 10101010   * * * *                  

lowerPlanetSurfaceCharset                                                             
        .BYTE $AA,$00,$AA,$A2,$00,$08,$00,$00   ;.BYTE $AA,$00,$AA,$A2,$00,$08,$00,$00
                                                ; CHARACTER $c0                       
                                                ; 10101010   * * * *                  
                                                ; 00000000                            
                                                ; 10101010   * * * *                  
                                                ; 10100010   * *   *                  
                                                ; 00000000                            
                                                ; 00001000       *                    
                                                ; 00000000                            
                                                ; 00000000                            
"""

In [22]:
import re
colors = {
    "00": "",
    "01": "\\BACKGROUNDONE",
    "10": "\\BACKGROUNDTWO",
    "11": "\\CHARCOLOR",
}
charsets = {}
charset_data = []
curr_charset = []
for l in charset_lines.split('\n'):
    if "CHARACTER" in l or "Planet" in l:
        if charset_data:
            curr_charset += [charset_data]
        charset_data = []
    if "Planet" in l:
        if curr_charset:
            charsets[charset_name] = curr_charset
        charset_name = l.strip()
        curr_charset = []
        continue

    m = re.findall(r"[0-1]{8}",l)
    if not m:
        continue
    bits = m[0]
    for i in range(0,7,2):
        bitpair = bits[i:i+2]
        charset_data += [colors[bitpair]]
        charset_data += [colors[bitpair]]
charsets[charset_name] = [charset_data]
charsets

{'upperPlanetSurfaceCharset': [['',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '',
   '',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO']],
 'lowerPlanetSurfaceCharset': [['\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',
   '\\BACKGROUNDTWO',


In [15]:
def fillCharacter(c, i=0, BIT_WIDTH=8):
    """
    Create a latex output that fills the cells in an 8x8 cell
    in the appropriate position in the grid.
    """
    output = ""
    gh = BIT_WIDTH - 1
    for j,v in enumerate(c):
        if not v:
            continue
        x = (i * BIT_WIDTH) + ((j % BIT_WIDTH) if j >= (BIT_WIDTH - 1) else j)
        y = (gh - int(j/BIT_WIDTH)) if j >= gh else gh
        output += f'\t\\fill[{v}] ({x},{y}) rectangle ++ (1,1);\n'
    return output

In [19]:
planet_colors = {
    'lowerPlanetSurfaceCharset' : ['brown','lightblue','lightgreen'],
    'upperPlanetSurfaceCharset' : ['brown','lightblue','lightgreen'],
    'planet1Charset' : ['brown','lightblue','lightgreen'],
    'planet2Charset' : ['gray','lightred','brown'],
    'planet3Charset' : ['yellow','white', 'lightred'],
    'planet4Charset' : ['lightblue','yellow', 'darkgray'],
    'planet5Charset' : ['lightgreen','lightred','lightred'],
}

In [23]:
def createTile(planet_name, char, index, ref):
    """
    Create a tile in the tilesheet.
    """
    WIDTH = "0.5cm"
    header = f"""
    \\begin{{figure}}[H]
      {{
        \\setlength{{\\tabcolsep}}{{3.0pt}}
        \\setlength\cmidrulewidth{{\lightrulewidth}} % Make cmidrule = 
        \\begin{{adjustbox}}{{width={WIDTH},center}}
          \\begin{{tikzpicture}}
    """
    footer = f"""
          \end{{tikzpicture}}
        \end{{adjustbox}}
      }}\caption*{{\\{ref}}}
    \end{{figure}}
    """

    BIT_WIDTH = 8
    output = header + '\n'
    output += f"\\def\BACKGROUNDONE{{{planet_colors[planet_name][0]}}}\n"
    output += f"\\def\BACKGROUNDTWO{{{planet_colors[planet_name][1]}}}\n"
    output += f"\\def\CHARCOLOR{{{planet_colors[planet_name][2]}}}\n"
    
    grid = f"\\draw[step=1.0,gray,thin] (0,0) grid ({BIT_WIDTH},{BIT_WIDTH});"
    output += f"\t{grid}\n"

    output += fillCharacter(char)

    output += footer
    return output



Create a bunch of files of the form `planetCharset1_tilesheet.tex` that
contain a figure with the full tilesheet for that planet.

In [24]:
header = """
\\begin{figure}[H]                          
{                                          
  \\setlength{\\tabcolsep}{3.0pt}            
  \\setlength\cmidrulewidth{\lightrulewidth}
"""
for planet_name, charset in charsets.items():
    figure = header
    for i, c in enumerate(charset):
        ref = f"${hex(0x40 + i)[2:].upper()}"
        output = createTile(planet_name, c, i, ref)
        of_name = f"../src/planets/tilesheets/{planet_name}_tilesheet_{ref}_inv.tex"
        with open(of_name,'w') as output_file:
            output_file.write(output)

        figure += "\\begin{subfigure}{0.12\\textwidth}\n"
        figure += f"\\input{{tilesheets/{planet_name}_tilesheet_{ref}_inv}}\n"
        figure += "\\end{subfigure}\n"

    footer = f"""
    }}\\caption[]{{Tilesheet: {planet_name}}}
    \\end{{figure}}
    """
    figure += footer
    of_name = f"../src/planets/{planet_name}_tilesheet_inv.tex"
    with open(of_name,'w') as output_file:
        output_file.write(figure)
